In [7]:
import random

import pandas as pd
import os
import numpy as np
import play_othello
from collections import defaultdict

move_dict = {
    'a': 0,
    'b': 1,
    'c': 2,
    'd': 3,
    'e': 4,
    'f': 5,
    'g': 6,
    'h': 7
}

In [8]:
data = pd.read_csv('othello_dataset.csv')
data.head()

,eOthello_game_id,winner,game_moves
0,1056798,1,f5d6c4d3e6f4e3f6c5b4e7f3c6d7b5a5c3b3g5h5g4h4e2...
1,1050515,-1,d3c5f6f5e6e3d6f7b6d7e2c6d8c4e7c3f4c8c7d2d1a6b3...
2,996989,1,f5d6c4d3c3f4e6b3e2c5c7f6g5e3e7g6g4h4f3g3f2h6h5...
3,977775,0,f5d6c5b4c3e3e2f4f3g4e6g5f6f7e7g3h6c4d3h5g6h7b3...
4,900860,1,f5d6c4d3c3f4c5b4g3c6b5f6e2g4e3e6b3a5d7e7f3a3h4...


In [9]:
def get_move_label(x,y):
    """
    Converts a move in the form of (x,y) to a label from 0-59
    :param x: row # of the move
    :param y: column # of the move
    :return: a label representing the move
    """

    encoding = 8*x + y

    if encoding <= 26:
        return encoding
    elif 29 <= encoding <= 34:
        return encoding - 2
    return encoding - 4

def get_coord_after_rotation(x,y,rotation):

    if rotation == 90:
        new_x, new_y = 7-y, x
    elif rotation == 180:
        new_x, new_y = 7-x, 7-y
    elif rotation == 270:
        new_x, new_y = y, 7-x
    return new_x, new_y

def get_coord_after_reflection(x,y,reflection):

    if reflection == 'horizontal':
        new_x, new_y = x, 7 - y
    elif reflection == 'vertical':
        new_x, new_y = 7-x, y
    elif reflection == 'diagonal':
        new_x, new_y = y,x
    return new_x, new_y

In [10]:
def make_board_matrices(board, tile):

    opp_tile = int(not bool(tile))
    player_positions = np.argwhere(board == tile)
    opponent_positions = np.argwhere(board == opp_tile)

    # Make player and opponent matrices
    player_matrix = np.zeros(shape=(8,8))
    player_matrix[player_positions[:,0], player_positions[:,1]] = 1

    opponent_matrix = np.zeros(shape=(8,8))
    opponent_matrix[opponent_positions[:,0], opponent_positions[:,1]] = 1

    # Stack matrices together and add regularization matrix
    board_matrix = np.array([player_matrix, opponent_matrix, np.ones((8,8))])

    return board_matrix

In [11]:
def get_player_board(board, tile):

    board_copy = np.array(board)
    opp_tile = int(not bool(tile))
    player_positions = np.argwhere(board == tile)
    opponent_positions = np.argwhere(board == opp_tile)

    board_copy[player_positions[:,0], player_positions[:,1]] = 1
    board_copy[opponent_positions[:,0], opponent_positions[:,1]] = 0

    return board_copy

In [ ]:
# Maps tuple version of board to the array representation of the board
board_dict = defaultdict(set)
game_idx = 0

for move_list in data.game_moves:

    # Create a list of coordinate moves
    moves = [move_list[i:i+2] for i in range(0, len(move_list), 2)]
    moves = [(int(move[1]) - 1, move_dict[move[0]]) for move in moves]

    # Initialize board
    board = np.ones((8,8)) - 2
    board[3][3] = 0
    board[4][4] = 0
    board[3][4] = 1
    board[4][3] = 1

    tile = 1

    for move_x, move_y in moves:

        # If the current player can't make a move then switch players
        if len(play_othello.getValidMoves(board, tile)) == 0:
            tile = int(not bool(tile))

        rot_90_board = np.rot90(board)
        rot_180_board = np.rot90(board, k=2)
        rot_270_board = np.rot90(board, k=3)
        hflip_board = np.fliplr(board)
        vflip_board = np.flipud(board)
        diagonal_flip_board = np.array([board[j][i] for i in range(board.shape[0]) for j in range(board.shape[1])]).reshape((8,8))

        # Convert player board to tuple
        tuple_board = tuple(list(get_player_board(board,tile).flatten()))
        tuple_rot_90_board = tuple(list(get_player_board(rot_90_board,tile).flatten()))
        tuple_rot_180_board = tuple(list(get_player_board(rot_180_board,tile).flatten()))
        tuple_rot_270_board = tuple(list(get_player_board(rot_270_board,tile).flatten()))
        tuple_hflip_board = tuple(list(get_player_board(hflip_board,tile).flatten()))
        tuple_vflip_board = tuple(list(get_player_board(vflip_board,tile).flatten()))
        tuple_dflip_board = tuple(list(get_player_board(diagonal_flip_board,tile).flatten()))

        # Get moves for rotated/reflected boards
        move_rot_90_x, move_rot_90_y = get_coord_after_rotation(move_x, move_y, 90)
        move_rot_180_x, move_rot_180_y = get_coord_after_rotation(move_x, move_y, 180)
        move_rot_270_x, move_rot_270_y = get_coord_after_rotation(move_x, move_y, 270)

        move_hflip_x, move_hflip_y = get_coord_after_reflection(move_x, move_y, 'horizontal')
        move_vflip_x, move_vflip_y = get_coord_after_reflection(move_x, move_y, 'vertical')
        move_dflip_x, move_dflip_y = get_coord_after_reflection(move_x, move_y, 'diagonal')


        # Add the move the player made for the set of moves made for this board state
        board_dict[tuple_board].add(get_move_label(move_x, move_y))
        board_dict[tuple_rot_90_board].add(get_move_label(move_rot_90_x, move_rot_90_y))
        board_dict[tuple_rot_180_board].add(get_move_label(move_rot_180_x, move_rot_180_y))
        board_dict[tuple_rot_270_board].add(get_move_label(move_rot_270_x, move_rot_270_y))
        board_dict[tuple_hflip_board].add(get_move_label(move_hflip_x, move_hflip_y))
        board_dict[tuple_vflip_board].add(get_move_label(move_vflip_x, move_vflip_y))
        board_dict[tuple_dflip_board].add(get_move_label(move_dflip_x, move_dflip_y))

        # Get the pieces that are flipped after the move
        flipped_pieces = np.array(play_othello.isValidMove(board, tile, move_x, move_y))

        # Make the move and flip the pieces
        board[move_x][move_y] = tile
        board[flipped_pieces[:,0], flipped_pieces[:,1]] = tile

        # Switch player turn
        tile = int(not bool(tile))

    game_idx +=1

In [ ]:
tuple_board_states = list(board_dict.keys())
random.shuffle(tuple_board_states)
len(tuple_board_states)

In [15]:

board_states = []

# Contains all the labels for the
labels = []

for tuple_board_state in tuple_board_states:


    # Reshape the board array to a 2D matrix
    board = np.reshape(np.array(list(tuple_board_state)), (8,8))

    # Get the player representation of the matrix,
    # should be 3x8x8
    board_matrix = make_board_matrices(board, 1)

    for move_label in board_dict[tuple_board_state]:

        if board_states is None:
            board_states = board_matrix
        else:
            board_states.append(board_matrix)

        labels.append(move_label)


In [ ]:
labels = np.array(labels)
board_states = np.array(board_states)
labels.shape, board_states.shape

In [ ]:
train_data = board_states[:int(0.8*board_states.shape[0])]
train_labels = labels[:int(0.8*board_states.shape[0])]

val_data = board_states[int(0.8*board_states.shape[0]):int(0.9*board_states.shape[0])]
val_labels = labels[int(0.8*board_states.shape[0]):int(0.9*board_states.shape[0])]

test_data = board_states[int(0.9*board_states.shape[0]):]
test_labels = labels[int(0.9*board_states.shape[0]):]

train_data.shape, train_labels.shape, val_data.shape, val_labels.shape, test_data.shape, test_labels.shape

In [ ]:
with open('train_data_large.npy', 'wb') as f:
    np.save(f, train_data)

with open('train_labels_large.npy', 'wb') as f:
    np.save(f, train_labels)

with open('val_data_large.npy', 'wb') as f:
    np.save(f, val_data)

with open('val_labels_large.npy', 'wb') as f:
    np.save(f, val_labels)

with open('test_data_large.npy', 'wb') as f:
    np.save(f, test_data)

with open('test_labels_large.npy', 'wb') as f:
    np.save(f, test_labels)